In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, double_sha256, little_endian_to_int, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

passphrase = b''
secret = little_endian_to_int(double_sha256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
target_amount = 0.1
fee = 0.005

response = requests.get('https://testnet.blockexplorer.com/api/addr/{}/utxo'.format(my_address))
utxos = response.json()

if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = 'mtShBfBH9LFyHmqJFuKjmAPfv6jJmwpbeV'
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['amount']
    # create a new tx input
    tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        ))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount * 100000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = int((total-fee)*100000000) - output_amount
h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv, SIGHASH_ALL)

# SANITY CHECK: fee is reasonable
if tx_obj.fee(testnet=True) > 0.05*100000000 or tx_obj.fee(testnet=True) <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, double_sha256, little_endian_to_int, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

passphrase = b''

secret = little_endian_to_int(double_sha256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.005

response = requests.get('https://testnet.blockexplorer.com/api/addr/{}/utxo'.format(my_address))
utxos = response.json()

student_addresses = 'mtShBfBH9LFyHmqJFuKjmAPfv6jJmwpbeV'
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['amount']
    # create a new tx input
    tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        ))

# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = randint(5000000, 20000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = int((total-fee)*100000000) - output_amount

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv, SIGHASH_ALL)

# SANITY CHECK: fee is reasonable
if tx_obj.fee(testnet=True) > 0.05*100000000 or tx_obj.fee(testnet=True) <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())